In [3]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': ['./Data/mednli_train.csv' ], 'test': './Data/mednli_dev.csv'})


Using custom data configuration default-9bd58d32244e627e


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to C:\Users\Subha\.cache\huggingface\datasets\csv\default-9bd58d32244e627e\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'sentence_1', 'sentence_2'],
        num_rows: 11232
    })
    test: Dataset({
        features: ['label', 'sentence_1', 'sentence_2'],
        num_rows: 1395
    })
})

In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments

In [18]:
dataset["train"][0]

{'label': 'entailment',
 'sentence_1': ' Labs were notable for Cr 1.7 (baseline 0.5 per old records) and lactate 2.4. sentence2: Patient has elevated Cr',
 'sentence_2': ' Patient has elevated Cr'}

In [27]:
model_name = "microsoft/deberta-v3-small"
model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels = 3)
tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast=True)

loading configuration file https://huggingface.co/microsoft/deberta-v3-small/resolve/main/config.json from cache at C:\Users\Subha/.cache\huggingface\transformers\8e0c12a7672d1d36f647c86e5fc3a911f189d8704e2bc94dde4a1ffe38f648fa.9df96bac06c2c492bc77ad040068f903c93beec14607428f25bf9081644ad0da
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-small",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidde

In [28]:
metric_name = 'accuracy'

In [40]:
def preprocess_function(examples):
    return tokenizer(examples["sentence_1"], examples["sentence_2"], truncation=True,max_length=512)

encoded_dataset = dataset.map(preprocess_function)

  0%|          | 0/11232 [00:00<?, ?ex/s]

  0%|          | 0/1395 [00:00<?, ?ex/s]

In [43]:
encoded_dataset = encoded_dataset.remove_columns(dataset["train"].column_names)

In [44]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 11232
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1395
    })
})

In [45]:
# Finetune the model on dataset for NLI task
from datasets import load_metric

task = "mnli"
metric = load_metric('glue', task)
batch_size = 8
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    # push_to_hub=True,
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
   
    predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 11232
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7020


  0%|          | 0/7020 [00:00<?, ?it/s]

KeyError: 'loss'